edwith의 부스트 코스 : "파이토치로 시작하는 딥러닝 기초" 를 바탕으로 작성되었습니다.  
https://www.boostcourse.org/ai214

# Multivariate Linear Regression
- Multivariate Linear Regression 이론
- Naive Data Representation
- Matrix Data Representation
- Multivariate Linear Regression 
- nn.Module
- F.mse_loss

## Multivariate Linear Regression 이론
쪽지 시험별 성적이 각각 다른 학생의 기말고사 점수를 예측하는 모델

### dataset 
다섯명의 학생의 3번의 쪽지시험 점수가 입력으로, 기말고사 점수가 출력으로 주어짐. 

In [3]:
import torch

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90], 
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


### Hypothesis Function 
simple linear regression 처럼 동일하게 $W * x + b$ 의 형태를 취하는데 x 를 1 * 1 벡터로 표현했다면  
현재의 x 값은 하나의 값이 아니라 여러값이기 때문에 3 * 1 벡터로 표현할 수 있을 것임.  
$$H(x) = w_1x_1 + w_2x_2 + w_3x_3 + b$$

위의 수식처럼 표현이 가능한데, 이보다 더 많은 양의 x를 가지고 있다면 이를 코드로 구현하는 것은 불가능에 가까움.  
그렇기 때문에 PyTorch 에서 제공하는 `matmul()` 함수를 이용함. matmul = matrix multipication의 줄임말.  

### Cost function : MSE 
simple linear regression 과 마찬가지로 MSE를 사용하며 계산방식 역시 동일

### Gradient Descent 
학습방식 역시 동일함. optimizer 를 정의한 후 cost를 구할때마다 optimizer의 gradient 에 저장한 후 gradient descent 를 시행함. 

### Full Code

In [7]:
import torch
import torch.optim as optim

W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20

for epoch in range(1, nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b  # matmul 로 
    
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    print('Epoch : {:4d}/{} hypothesis: {} Cost : {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

Epoch :    1/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost : 29661.800781
Epoch :    2/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost : 9298.520508
Epoch :    3/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost : 2915.712646
Epoch :    4/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost : 915.040527
Epoch :    5/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost : 287.936005
Epoch :    6/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost : 91.371017
Epoch :    7/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost : 29.758139
Epoch :    8/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost : 10.445305
Epoch :    9/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost : 4.391228
Epoch :   10/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost

## nn.Module
nn.Modul 을 사용하여 클래스를 만들면 모델을 보다 쉽게 만들 수 있음. 

```python
import torch.nn as nn

class Multivariate_Linear_RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
    
model = Multivariate_Linear_RegressionModel()
    
hypothesis = model(x_train)
```

nn.Module 를 상속해서 모델을 생성,  
nn.Linear(3, 1) 입력차원과 출력차원을 입력  
Hypothesis 계산은 forward 함수에서 진행되며, 
gradient 계산은 PyTroch에서 자동으로 진행해줌.  

## F.mse_loss 
PyTorch 에서는 다양한 cost function 을 제공함.  
PyTorch 의 cost function 을 사용하면 다음 cost function 을 바꿀때 좀 더 편리하고, cost function 을 계산하면서 생기는 버그가 없어 디버깅이 쉽다는 장점이 있음.  
mse는 torch.nn.functional 에 존재하는 mse_loss 를 사용하면 됨.  

```python
import torch.nn.functional as F

cost = F.mse_loss(prediction, y_train)
```

In [18]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

class Multivariate_Linear_RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
    
model = Multivariate_Linear_RegressionModel()
    

# W = torch.zeros((3, 1), requires_grad=True)
# b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20

for epoch in range(1, nb_epochs + 1):
    # H(x) 계산
#     hypothesis = x_train.matmul(W) + b
    hypothesis = model(x_train)
    
    # Cost 계산
#     cost = torch.mean((hypothesis - y_train) ** 2)
    cost = F.mse_loss(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch : {:4d}/{} hypothesis: {} Cost : {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))

Epoch :    1/20 hypothesis: tensor([-9.3751, -4.1805, -7.8001, -8.0960, -1.7647]) Cost : 31884.699219
Epoch :    2/20 hypothesis: tensor([60.3576, 79.6319, 74.7822, 81.8341, 62.1629]) Cost : 9995.871094
Epoch :    3/20 hypothesis: tensor([ 99.3986, 126.5553, 121.0172, 132.1826,  97.9533]) Cost : 3134.886475
Epoch :    4/20 hypothesis: tensor([121.2566, 152.8257, 146.9025, 160.3710, 117.9909]) Cost : 984.331177
Epoch :    5/20 hypothesis: tensor([133.4942, 167.5334, 161.3948, 176.1526, 129.2090]) Cost : 310.245728
Epoch :    6/20 hypothesis: tensor([140.3459, 175.7675, 169.5086, 184.9883, 135.4893]) Cost : 98.954575
Epoch :    7/20 hypothesis: tensor([144.1822, 180.3773, 174.0513, 189.9351, 139.0052]) Cost : 32.725243
Epoch :    8/20 hypothesis: tensor([146.3302, 182.9580, 176.5946, 192.7047, 140.9734]) Cost : 11.965099
Epoch :    9/20 hypothesis: tensor([147.5331, 184.4027, 178.0186, 194.2553, 142.0752]) Cost : 5.457086
Epoch :   10/20 hypothesis: tensor([148.2068, 185.2113, 178.8160, 